In [1]:
# pip install --upgrade nbformat

# Imports
## NOTE:
You will need to have installed the Google Cloud SDK and a project set up to run this work flow interactively. Instructions on how to set up the Google Cloud SDK are [here](https://cloud.google.com/sdk/docs/install).

In [2]:
import ee

from gee_water.utils import (
    add_ndvi_l30,
    add_savi_l30,
    create_monthly_composites,
    get_time_series,
    get_monthly_image,
    export_gee_image_to_local_png_with_palette
)
from gee_water.visualization_utils import plot_multiple_timeseries_with_trendlines_plotly, create_split_map
from gee_water.analysis import check_trend_significance
from gee_water.masks import mask_hls_l30_cloud_shadow
from gee_water.vis_params import ndvi_vis_params

from functools import partial



In [3]:
ee.Initialize()

# README
The purpose of this notebook is to do some exploratory work on NDVI for regions in Southern Baja for a blog post exploring water resources in the area.
The focus region for this project will be Baja California Sur, near the southern most tip of the peninsula in the La Paz - Cabo - Todos Santos triangle.

## UPDATE - What is NDVI?
NDVI has no unit. Rather, it is an index value in which higher values (0.4 to 0.9) show lands covered by green, leafy vegetation and lower values (0 to 0.4) show lands where there is little or no vegetation. Negative values tend to indicate the presence of water, clouds or snow.

## Define Region of Interest

In [4]:
# Coordinates for La Paz, BCS
la_paz_coords = ee.Geometry.Point([-110.3166667, 24.1422222])
buffer_distance_meters = 100 * 1609.34  # 75 miles in meters
roi = la_paz_coords.buffer(buffer_distance_meters)

# Run NDVI Analysis Using HLS GEE Data Set

In [5]:
# For this particular blog, I am analyzing multiple datasets from a handful of different 
# satellites. The year 2013 is far enough back historically to capture form trends, but still 
# allows for higih quality data to be gathred for all my metrics of interest.
start_date = '2013-01-01'
end_date   = '2023-12-31'


In [6]:
hls_l30_with_ndvi = (
    ee.ImageCollection('NASA/HLS/HLSL30/v002')
    .filterDate(start_date, end_date)
    .filterBounds(roi)
    .map(mask_hls_l30_cloud_shadow)
    .map(add_ndvi_l30)
)


In [7]:
add_savi_l30_partial = partial(add_savi_l30, L=0.8)


hls_l30_with_savi = (
    ee.ImageCollection('NASA/HLS/HLSL30/v002')
    .filterDate(start_date, end_date)
    .filterBounds(roi)
    .map(mask_hls_l30_cloud_shadow)
    .map(add_savi_l30_partial)
)


In [8]:
# create composites
hls_l30_ndvi_monthly = create_monthly_composites(hls_l30_with_ndvi)
hls_l30_savi_monthly = create_monthly_composites(hls_l30_with_savi)


In [9]:
# create time series data from the monthly composites
hls_l30_ndvi_timeseries = get_time_series(hls_l30_ndvi_monthly, 'NDVI', roi)
hls_l30_savi_timeseries = get_time_series(hls_l30_savi_monthly, 'SAVI', roi)

In [10]:
plot_multiple_timeseries_with_trendlines_plotly(
    df_list=[hls_l30_ndvi_timeseries],
    date_col='date',
    value_col='NDVI',
    labels=['HLS NDVI'],
    x_interval_months=6,
    title='HLS Derived NDVI 2013 - 2023',
    ylabel='NDVI (-)',
    html_export_location='./../local_images/navi_trend_chart.html'
)

In [11]:
plot_multiple_timeseries_with_trendlines_plotly(
    df_list=[hls_l30_savi_timeseries],
    date_col='date',
    value_col='SAVI',
    labels=['HLS SAVI'],
    x_interval_months=6,
    title='HLS Derived SAVI 2013 - 2023',
    ylabel='SAVI (-)',
    html_export_location='./../local_images/savi_trend_chart.html'
)

# Determine if Trend is Statistically Significant

In [12]:
# Check the trend significance
slope_day, slope_year, p_val, is_sig = check_trend_significance(
    hls_l30_ndvi_timeseries,
    date_col='date',
    value_col='NDVI',
    alpha=0.05
)

print("Significance Test")
print("----------------------")
print(f"Slope (units/day): {slope_day:.5f}")
print(f"Slope (units/year): {slope_year:.5f}")
print(f"P-value: {p_val:.5g}")
print(f"Significant at alpha=0.05? {is_sig}")

Significance Test
----------------------
Slope (units/day): -0.00002
Slope (units/year): -0.00580
P-value: 0.21526
Significant at alpha=0.05? False


In [13]:
# Check the trend significance
slope_day, slope_year, p_val, is_sig = check_trend_significance(
    hls_l30_savi_timeseries,
    date_col='date',
    value_col='SAVI',
    alpha=0.05
)

print("Significance Test")
print("----------------------")
print(f"Slope (units/day): {slope_day:.5f}")
print(f"Slope (units/year): {slope_year:.5f}")
print(f"P-value: {p_val:.5g}")
print(f"Significant at alpha=0.05? {is_sig}")

Significance Test
----------------------
Slope (units/day): 0.00000
Slope (units/year): 0.00009
P-value: 0.82616
Significant at alpha=0.05? False


# Display NDVI Year to Year

In [14]:
# June early NDVI
hls_ndvi_june_2013 = get_monthly_image(hls_l30_with_ndvi, 'NDVI', 2013, 6)
hls_savi_june_2013 = get_monthly_image(hls_l30_with_savi, 'SAVI', 2013, 6)


# June 2023 NDVI
hls_ndvi_june_2023 = get_monthly_image(hls_l30_with_ndvi, 'NDVI', 2023, 6)
hls_savi_june_2023 = get_monthly_image(hls_l30_with_savi, 'SAVI', 2023, 6)


# December early NDVI
hls_ndvi_dec_2013 = get_monthly_image(hls_l30_with_ndvi, 'NDVI', 2013, 12)
hls_savi_dec_2013 = get_monthly_image(hls_l30_with_savi, 'SAVI', 2013, 12)

# December 2023 NDVI
hls_ndvi_dec_2023 = get_monthly_image(hls_l30_with_ndvi, 'NDVI', 2023, 12)
hls_savi_dec_2023 = get_monthly_image(hls_l30_with_savi, 'SAVI', 2023, 12)



In [15]:
type(hls_ndvi_june_2013)

ee.image.Image

In [16]:
create_split_map(
        left_image=hls_ndvi_june_2013, 
        right_image=hls_ndvi_june_2023,
        left_image_label="HLS NDVI June 2013",
        right_image_label="HLS NDVI June 2023",
        vis_params=ndvi_vis_params,
        mep_center_coords=[24.1422222, -110.3166667],
        zoom=10,
        legend_title='NDVI',
        legend_labels= ['Low (0)', 'Mid (0.5)', 'High (1)'],
        html_export_location='./../static/html/ndvi_june_comparison.html'
)

Map(center=[24.1422222, -110.3166667], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_tit…

In [17]:
create_split_map(
        left_image=hls_savi_june_2013, 
        right_image=hls_savi_june_2023,
        left_image_label="HLS SAVI June 2013",
        right_image_label="HLS SAVI June 2023",
        vis_params=ndvi_vis_params,
        mep_center_coords=[24.1422222, -110.3166667],
        zoom=10,
        legend_title='NDVI',
        legend_labels= ['Low (0)', 'Mid (0.5)', 'High (1)'],
        html_export_location='./../static/html/savi_june_comparison.html'
)

Map(center=[24.1422222, -110.3166667], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_tit…

# Export Images Locally

In [18]:
export_gee_image_to_local_png_with_palette(
    ee_image=hls_savi_june_2013,
    roi=roi,
    tif_file_path="../local_images/hls_savi_june_2013.tif",
    png_file_path="../local_images/hls_savi_june_2013.png",
    vis_params=ndvi_vis_params,
    epsg_coordinate_system='EPSG:4326'
)

Generating URL ...
Please wait ...
Data downloaded to /Users/alextruby/repos/gee-water/local_images/hls_savi_june_2013.tif
If using this in leaflet, image bounds are:  [[22.694549130472073, -111.88845802008942], [25.590445693035743, -108.73582798145047]]
../local_images/hls_savi_june_2013.tif is already in EPSG:4326


In [20]:
export_gee_image_to_local_png_with_palette(
        ee_image=hls_savi_june_2023,
        roi=roi,
        tif_file_path='../local_images/hls_savi_june_2023.tif',
        png_file_path='../local_images/hls_savi_june_2023.png',
        vis_params=ndvi_vis_params,
        epsg_coordinate_system='EPSG:4326'
)

Generating URL ...
Please wait ...
Data downloaded to /Users/alextruby/repos/gee-water/local_images/hls_savi_june_2023.tif
If using this in leaflet, image bounds are:  [[22.694549130472073, -111.88845802008942], [25.590445693035743, -108.73582798145047]]
../local_images/hls_savi_june_2023.tif is already in EPSG:4326


In [21]:
export_gee_image_to_local_png_with_palette(
        ee_image=hls_savi_dec_2013,
        roi=roi,
        tif_file_path='../local_images/hls_savi_dec_2013.tif',
        png_file_path='../local_images/hls_savi_dec_2013.png',
        vis_params=ndvi_vis_params,
        epsg_coordinate_system='EPSG:4326'
)

Generating URL ...
Please wait ...
Data downloaded to /Users/alextruby/repos/gee-water/local_images/hls_savi_dec_2013.tif
If using this in leaflet, image bounds are:  [[22.694549130472073, -111.88845802008942], [25.590445693035743, -108.73582798145047]]
../local_images/hls_savi_dec_2013.tif is already in EPSG:4326


In [23]:
export_gee_image_to_local_png_with_palette(
        ee_image=hls_savi_dec_2023,
        roi=roi,
        tif_file_path='../local_images/hls_savi_dec_2023.tif',
        png_file_path='../local_images/hls_savi_dec_2023.png',
        vis_params=ndvi_vis_params,
        epsg_coordinate_system='EPSG:4326'
)

Generating URL ...
Please wait ...
Data downloaded to /Users/alextruby/repos/gee-water/local_images/hls_savi_dec_2023.tif
If using this in leaflet, image bounds are:  [[22.694549130472073, -111.88845802008942], [25.590445693035743, -108.73582798145047]]
../local_images/hls_savi_dec_2023.tif is already in EPSG:4326
